<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/biframenodevisualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NOTE: After running this cell for the first time,
# you may need to refresh or restart the runtime so that widget extensions load properly.
# Once you've done that, re-run the cell and the visualization should appear.

# Install dependencies
!pip install rdflib ipycytoscape

# Enable ipywidgets and ipycytoscape in Colab
!pip install jupyterlab==3.0.0 ipywidgets==7.6.3
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py ipycytoscape --sys-prefix

from google.colab import files
from rdflib import Graph
from ipycytoscape import CytoscapeWidget, Node, Edge
from IPython.display import display, HTML
from google.colab import output

# Enable the custom widget manager for Google Colab
output.enable_custom_widget_manager()

print("Please upload your Turtle (.ttl) file:")
uploaded = files.upload()
ttl_file = list(uploaded.keys())[0]  # Use the first uploaded file

# Parse the Turtle file
g = Graph()
g.parse(ttl_file, format="turtle")

print(f"Parsed {len(g)} triples from {ttl_file}")

# Create a CytoscapeWidget instance
cyto_graph = CytoscapeWidget()

# Prepare nodes and edges
nodes = set()
edges = []

for s, p, o in g:
    s_str, p_str, o_str = str(s), str(p), str(o)
    nodes.add(s_str)
    nodes.add(o_str)
    edges.append((s_str, o_str, p_str))

def classify_node(uri):
    uri_lower = uri.lower()
    if "work/" in uri_lower:
        return "work"
    elif "instance/" in uri_lower:
        return "instance"
    elif "person" in uri_lower or "agent" in uri_lower:
        return "person"
    elif "provision" in uri_lower or "place" in uri_lower:
        return "place"
    else:
        return "other"

node_list = []
for n in nodes:
    node_class = classify_node(n)
    # Create a Node object
    node_obj = Node(data={'id': n, 'label': n}, classes=node_class)
    node_list.append(node_obj)

edge_list = []
for (source, target, pred) in edges:
    edge_obj = Edge(data={'source': source, 'target': target, 'label': pred}, classes='relationship')
    edge_list.append(edge_obj)

# Add nodes and edges to the graph
for node in node_list:
    cyto_graph.graph.add_node(node)

for edge in edge_list:
    cyto_graph.graph.add_edge(edge)

# Apply visually striking style
cyto_graph.set_style([
    {
        'selector': 'node',
        'style': {
            'background-color': '#1ABC9C',
            'color': 'white',
            'label': 'data(label)',
            'font-family': 'Helvetica',
            'font-size': '10px',
            'text-outline-width': '2px',
            'text-outline-color': '#0E6251',
            'text-halign': 'center',
            'text-valign': 'center',
            'width': '45px',
            'height': '45px',
            'overlay-padding': '6px',
            'border-color': '#0E6251',
            'border-width': '2px',
            'border-opacity': '0.8'
        }
    },
    {
        'selector': 'node.work',
        'style': {
            'background-color': '#F1C40F',
            'shape': 'hexagon'
        }
    },
    {
        'selector': 'node.instance',
        'style': {
            'background-color': '#2E86C1',
            'shape': 'ellipse'
        }
    },
    {
        'selector': 'node.person',
        'style': {
            'background-color': '#E74C3C',
            'shape': 'round-rectangle'
        }
    },
    {
        'selector': 'node.place',
        'style': {
            'background-color': '#9B59B6',
            'shape': 'diamond'
        }
    },
    {
        'selector': 'node.other',
        'style': {
            'background-color': '#16A085',
            'shape': 'star'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#BDC3C7',
            'width': 2,
            'curve-style': 'bezier',
            'target-arrow-shape': 'triangle',
            'target-arrow-color': '#BDC3C7',
            'arrow-scale': 1.2,
            'font-size': '6px',
            'color': '#ECF0F1',
            'text-background-color': '#2C3E50',
            'text-background-opacity': 0.8,
            'text-background-padding': '2px',
            'text-rotation': 'autorotate',
            'label': 'data(label)',
        }
    },
    {
        'selector': ':selected',
        'style': {
            'background-color': '#F39C12',
            'line-color': '#F39C12',
            'target-arrow-color': '#F39C12',
            'source-arrow-color': '#F39C12'
        }
    }
])

# Use an aesthetically pleasing layout (fcose)
cyto_graph.set_layout(name='fcose',
                      animate=True,
                      randomize=True,
                      fit=True,
                      nodeSeparation=150,
                      idealEdgeLength=120,
                      gravity=0.8,
                      edgeElasticity=0.2)

# Show a title and instructions
display(HTML("""
<h2 style="text-align:center; font-family:Helvetica; margin-top:20px;">A Galaxy of Knowledge</h2>
<p style="text-align:center; max-width:600px; margin: 20px auto; font-family:Helvetica; line-height:1.4;">
  Explore your bibliographic data as a cosmic tapestry. Each node a star, each edge a delicate filament.
  Pan, zoom, and hover over nodes and edges to discover hidden connections. This is data as art.
</p>
"""))

display(cyto_graph)